In [159]:
import numpy as np
import cv2
from os import listdir
from os.path import isfile,join
from tqdm import tqdm
import tensorflow as tf
from keras.models import Sequential , load_model
from keras.layers import Dense,Dropout,Activation,Conv2D, MaxPool2D,Flatten
# import matplotlib.pyplot as plt
import keras
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [160]:
num_class = 3
mypath = 'dataset/animal/'
imgPath = [mypath + f for f in listdir(mypath)]
print(imgPath)

['dataset/animal/cat', 'dataset/animal/dog', 'dataset/animal/lion']


In [161]:
def img2data(imgPath):
    rawImgs = []
    labels = []
    for imagePath in imgPath:
        # print(imagePath)
        for item in listdir(imagePath):
            # print(item)
            file = imagePath + "/" + item
            # print(file,file[-1])
            # img = mpimg.imread(file)
            # plt.imshow(img)
            img = cv2.imread(file)
            img = cv2.resize(img,(128,128))
            rawImgs.append(img)
            i = imagePath.split('/')
            if i[2] == 'cat':
                labels.append([1,0,0])
            if i[2] == 'dog':
                labels.append([0,1,0])
            if i[2] == 'lion':
                labels.append([0,0,1])
    return rawImgs,labels
            
            

In [162]:
x_trian , y_train = img2data(imgPath)
x_trian = np.array(x_trian)
y_train = np.array(y_train)
x_train = x_trian.astype('float32')
x_train /= 255
x_train.shape,y_train.shape

((30, 128, 128, 3), (30, 3))

In [163]:
x_train[0]

array([[[0.32156864, 0.34117648, 0.38039216],
        [0.3254902 , 0.34509805, 0.38039216],
        [0.3254902 , 0.34509805, 0.38039216],
        ...,
        [0.32156864, 0.34117648, 0.3764706 ],
        [0.31764707, 0.3372549 , 0.37254903],
        [0.31764707, 0.3372549 , 0.37254903]],

       [[0.32156864, 0.34117648, 0.3764706 ],
        [0.32156864, 0.34117648, 0.3764706 ],
        [0.3254902 , 0.34509805, 0.38039216],
        ...,
        [0.31764707, 0.3372549 , 0.37254903],
        [0.32156864, 0.34117648, 0.3764706 ],
        [0.31764707, 0.3372549 , 0.37254903]],

       [[0.32941177, 0.34901962, 0.38431373],
        [0.3254902 , 0.34509805, 0.38039216],
        [0.3254902 , 0.34509805, 0.38039216],
        ...,
        [0.31764707, 0.3372549 , 0.37254903],
        [0.31764707, 0.3372549 , 0.37254903],
        [0.31764707, 0.3372549 , 0.37254903]],

       ...,

       [[0.7294118 , 0.7176471 , 0.6862745 ],
        [0.73333335, 0.72156864, 0.6901961 ],
        [0.7490196 , 0

In [164]:
model = Sequential()

model.add(Conv2D(128,(3,3),activation='relu',input_shape=(128,128,3)))
model.add(MaxPool2D(2))
model.add(Dense(16))
model.add(Dense(16))
model.add(Flatten())
model.add(Dense(num_class,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 128)     3584      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 128)      0         
 )                                                               
                                                                 
 dense (Dense)               (None, 63, 63, 16)        2064      
                                                                 
 dense_1 (Dense)             (None, 63, 63, 16)        272       
                                                                 
 flatten (Flatten)           (None, 63504)             0         
                                                                 
 dense_2 (Dense)             (None, 3)                 190515    
                                                        

In [180]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss = 'categorical_crossentropy',
    metrics= ['accuracy']
)
batch_size = 32
model.fit(x_train,y_train,batch_size=batch_size,epochs=5,validation_data=(x_train,y_train))

Epoch 1/5
1/1 [==============================] - 2s 2s/step - loss: 0.5039 - accuracy: 0.9333 - val_loss: 2.3625 - val_accuracy: 0.3667
Epoch 2/5
1/1 [==============================] - 1s 1s/step - loss: 2.3625 - accuracy: 0.3667 - val_loss: 0.4911 - val_accuracy: 0.8667
Epoch 3/5
1/1 [==============================] - 1s 1s/step - loss: 0.4911 - accuracy: 0.8667 - val_loss: 0.5801 - val_accuracy: 0.7333
Epoch 4/5
1/1 [==============================] - 1s 944ms/step - loss: 0.5801 - accuracy: 0.7333 - val_loss: 0.6349 - val_accuracy: 0.7333
Epoch 5/5
1/1 [==============================] - 1s 861ms/step - loss: 0.6349 - accuracy: 0.7333 - val_loss: 0.4763 - val_accuracy: 0.8333


In [181]:
model.save("trainFile.h5")

In [182]:
_model = keras.models.load_model('trainFile.h5')
path = "./dataset/animal/dog/8b46593423246eab1625f25aea55ea23.jpg"
img = tf.keras.preprocessing.image.load_img(
    path, target_size=(128,128)
)
print(img)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = _model.predict(img_array)

label = ['Cat','Dog','Lion']
result = label[np.argmax(predictions)]
print(np.argmax(predictions),'predict:'+str(result))

<PIL.Image.Image image mode=RGB size=128x128 at 0x2CB0DABA6A0>
1/1 [==============================] - 0s 417ms/step
1 predict:Dog


In [192]:
from tkinter import *
from tkinter.filedialog import *
from PIL import Image,ImageTk
import numpy as np
from tensorflow.keras.applications.imagenet_utils import preprocess_input

In [193]:
def chooseImage():
    fileTypes = (
        ('image file' , '*.jpg'),
        ('All files', '*.*')
    )
    fileName = askopenfilename()
    im = Image.open(fileName)
    resized_image = im.resize((300,300))
    ph = ImageTk.PhotoImage(resized_image)
    label = Label(root, image=ph)
    label.image = ph
    label.grid(row=1,column=0)
    predictImage(fileName)
        

In [194]:
def predictImage(fileName):
    loadModel = keras.models.load_model('trainFile.h5')
    labels = ["Cat","Dog","Lion"]
    img_path = fileName
    img = cv2.imread(img_path)
    img = cv2.resize(img, (128, 128))
    x = np.expand_dims(img, axis=0)
    x = preprocess_input(x)
    print('Input image shape:', x.shape)
    preds = loadModel.predict(x)
    print("predicted class: ", preds*100)  
    index_name = np.argmax(preds)
    print("Predict image is : ", labels[index_name])
    predictName.set(labels[index_name])
    predict_predict = preds[0][np.argmax(preds)]*100
    predictPercent.set(predict_predict)
    # _model = keras.models.load_model('trainFile.h5')
    # path = fileName
    # img = tf.keras.preprocessing.image.load_img(
    #     path, target_size=(128,128)
    # )
    # print(img)
    # img_array = tf.keras.preprocessing.image.img_to_array(img)
    # img_array = tf.expand_dims(img_array, 0)  # Create batch axis
    # predictions = _model.predict(img_array)
    # label = ['Cat','Dog','Lion']
    # result = label[np.argmax(predictions)]
    # animalName = str(result)
    # predictName.set(animalName)
    # predictPercent.set(predict_percent)
    # print(predictions*100)
    # print(animalName)

In [195]:
root = Tk()
root.title("Animal Predict")
root.geometry("400x400")
frame = Frame(root)
frame.grid(row=0,column=0)
btn1 = Button(frame,text = 'Choose file',command= chooseImage).grid(row=0,column=0)
predictName = StringVar()
predictPercent = StringVar()
Label(frame,text="Predict Name").grid(row=1,column=0)
e1 = Entry(frame,textvariable=predictName).grid(row=1,column=1)
Label(frame,text="Predict Percent").grid(row=2,column=0)
e2 = Entry(frame,textvariable=predictPercent).grid(row=2,column=1)
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\pomda\anaconda3\lib\site-packages\PIL\Image.py", line 3096, in open
    fp.seek(0)
AttributeError: 'str' object has no attribute 'seek'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\pomda\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\pomda\AppData\Local\Temp\ipykernel_1832\2035122172.py", line 7, in chooseImage
    im = Image.open(fileName)
  File "c:\Users\pomda\anaconda3\lib\site-packages\PIL\Image.py", line 3098, in open
    fp = io.BytesIO(fp.read())
AttributeError: 'str' object has no attribute 'read'


Input image shape: (1, 128, 128, 3)
1/1 [==============================] - 0s 185ms/step
predicted class:  [[ 0.3710636 99.62894    0.       ]]
Predict image is :  Dog
